In [ ]:
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from PIL import Image, ImageTk, ImageColor
from skimage import filters, segmentation, io, img_as_float, color
from skimage.measure import label, regionprops
from skimage.filters import sobel, apply_hysteresis_threshold

In [ ]:
window = tk.Tk()
window.title("TIF Image Viewer")

label_raw = tk.Label()
label_raw.grid(row=1,column=0)
label_label = tk.Label()
label_label.grid(row=1,column=1)
label_subimage = tk.Label()
label_subimage.grid(row=1,column=2)
# Load the image
# file_path = "../data/Marked 2nd.tif"
file_path = "../data/A1p150_R1,R2_induction_both.tif"

In [ ]:
image_raw = Image.fromarray(
    io.imread(file_path)
)
image_raw

In [ ]:
edges = sobel(sub_images[1][:,:,:3])
edges /= np.max(edges)

low = 1e-5
high = 1e-2

lowt = (edges > low).astype(int)
hight = (edges > high).astype(int)
hyst = apply_hysteresis_threshold(edges, low, high)

fig, ax = plt.subplots(nrows=2, ncols=2)

ax[0, 0].imshow(sub_images[1], cmap='gray')
ax[0, 0].set_title('Original image')

ax[0, 1].imshow(edges, cmap='magma')
ax[0, 1].set_title('Sobel edges')

ax[1, 0].imshow(lowt, cmap='magma')
ax[1, 0].set_title('Low threshold')

ax[1, 1].imshow(hight + hyst, cmap='magma')
ax[1, 1].set_title('Hysteresis threshold')

for a in ax.ravel():
    a.axis('off')

plt.tight_layout()

plt.show()

In [ ]:
plt.imshow(edges[:,:,0] < .1)

In [ ]:
plt.imshow(edges)
grayscale_image = color.rgb2gray(edges)
# threshold_value = np.median(edges)
threshold_value = filters.threshold_otsu(edges)
binary_image = grayscale_image > threshold_value
plt.imshow(binary_image)
plt.show()

In [ ]:
if file_path:
    image_raw = Image.fromarray(
        io.imread(file_path)
    )
    # image_raw = image_raw.resize((
    #     image_raw.size[0] //2,
    #     image_raw.size[1] //2
    # ))

    photo_raw = ImageTk.PhotoImage(image_raw)
    label_raw.configure(image=photo_raw)
    label_raw.image = photo_raw

In [ ]:
image_raw = img_as_float(image_raw)

alpha_label_image = label(image_raw[:, :, 3])
plt.imshow(alpha_label_image)

In [ ]:
sub_images = []

for region in regionprops(alpha_label_image):
    if region.area >= 25000:
        minr, minc, maxr, maxc = region.bbox
        sub_image = image_raw[minr:maxr,minc:maxc]
        sub_images.append(sub_image)

        plt.imshow(sub_image)
        plt.show()

        # grayscale_image = sub_image[:, :, 1]
        grayscale_image = color.rgb2gray(
            sub_image[:, :, :3]
        )
        plt.imshow(grayscale_image)
        plt.show()
        threshold_value = np.median(grayscale_image)
        # threshold_value = filters.threshold_otsu(grayscale_image)
        binary_image = grayscale_image > threshold_value
        plt.imshow(binary_image)
        plt.show()
        label_image = label(binary_image)
        
        # for sub_region in regionprops(label_image):
        #     sminr, sminc, smaxr, smaxc = sub_region.bbox
        #     sub_region_image = sub_image[sminr:smaxr,sminc:smaxc]
        

In [ ]:
grayscale_image = color.rgb2gray(
    img_as_float(image_raw)[:,:,:3]
)
threshold_value = filters.threshold_otsu(grayscale_image)
binary_image = grayscale_image > threshold_value
label_image = label(binary_image)
regions = iter([
    region
    for region
    in regionprops(label_image)
    if region.area >= 100
])

label_image = Image.fromarray(
    label_image.astype('uint8'),
    mode='P'
)

photo_label = ImageTk.PhotoImage(label_image)

label_label.configure(image=photo_label)
label_label.image = photo_label

In [ ]:
if label_image is not None:
    color_map = ImageColor.colormap
    color_map['gray'] = (128, 128, 128)
    color_map['red'] = (255, 0, 0)
    color_map['green'] = (0, 255, 0)
    color_map['blue'] = (0, 0, 255)

    color_image = label_image
    color_image.putpalette(
        sum(
            [
                list(color_map[name])
                for name
                in ('gray', 'red', 'green', 'blue')
            ],
            []
        )
    )

    photo_label = ImageTk.PhotoImage(color_image)
    label_label.configure(image=photo_label)
    label_label.image = photo_label

In [ ]:
color_image